# INIT

In [2]:
import os
import json
import boto3

with open(".env.json", "r") as f_in:
    env_json: dict = json.load(f_in)
    for key, value in env_json.items():
        os.environ[key] = value

aws_access_key = os.environ['aws_access_key']
aws_secret_key = os.environ['aws_secret_key']
aws_region = os.environ['aws_region']

# DEV

In [35]:
from langchain.llms.bedrock import Bedrock
from langchain.chains import LLMChain
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from bert_score import score

def lambda_handler(event, context):
   
    event_method = event["httpMethod"]
   
    if event_method != 'POST':
        return {
            'statusCode': 200,
            'body': json.dumps("Not implemented")
        }
       
    event_body = event["body"]
    prompt_data = event_body["prompt_input"]
    reference_output = event_body["respuesta_esperada"]

   
    client_bedrock_runtime = boto3.client(
        service_name="bedrock-runtime",
        region_name=aws_region,
        aws_access_key_id=aws_access_key,
        aws_secret_access_key=aws_secret_key,
        verify=False
    )
   
    model_id = 'amazon.titan-text-express-v1'
    
    titan_llm = Bedrock(
        model_id=model_id, 
        client=client_bedrock_runtime
    )

    titan_llm.model_kwargs = {
        'temperature': 0.5, 
        "maxTokenCount": 700,
        "topP": 0.9
    }

    chain = LLMChain(llm=titan_llm, prompt=rules_prompt, memory=memory, verbose=False) 
    chain_response = chain.run(prompt_data)
    #P, R, F1 = score(chain_response, reference_summaries, lang='es', verbose=True)
    P, R, F1 = score([chain_response], [reference_output], lang="es", verbose=True)

    return {
        'statusCode': 200,
        'body': json.dumps({
            'respuesta': chain_response,
            'precision': P.mean().item(),
            'recall': R.mean().item(),
            'f1': F1.mean().item(),
    })
    }

In [ ]:
test_input={"question": "Why should a victim go to the doctor after a Heimlich manoeuvre?"},
reference_output={"answer": "Because the Heimlich manoeuvre may have caused internal bleeding."},

In [54]:
event = {
    "httpMethod": "POST",
    "body": {
        "prompt_input": {prompt_input},
        "respuesta_esperada": {reference_output}
    }
}
lambda_response = lambda_handler(event, context=None)
print(lambda_response)
print(lambda_response.get('body'))

c:\Users\oriverac\AppData\Local\anaconda3\envs\env_ai\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'bedrock-runtime.us-east-1.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\oriverac\AppData\Local\anaconda3\envs\env_ai\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\oriverac\AppData\Local\anaconda3\envs\env_ai\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x

calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:00<00:00, 14.92it/s]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 1000.31it/s]

done in 0.07 seconds, 13.69 sentences/sec
{'statusCode': 200, 'body': '{"respuesta": "\\nLa cuota adicional por la inclusi\\u00f3n de nuevos Beneficiarios se agregar\\u00e1 al global de las siguientes cuotas del Contrato.", "precision": 0.9366027116775513, "recall": 0.9170214533805847, "f1": 0.9267086386680603}'}
{"respuesta": "\nLa cuota adicional por la inclusi\u00f3n de nuevos Beneficiarios se agregar\u00e1 al global de las siguientes cuotas del Contrato.", "precision": 0.9366027116775513, "recall": 0.9170214533805847, "f1": 0.9267086386680603}


# Postgres Initial Setup

In [3]:
import os
from PostgresHandler import PostgresHandler
configuration_db = {
    "postgresql": {
        "database": os.environ.get("postgres_database", None),
        "user": os.environ.get("postgres_user", None),
        "password": os.environ.get("postgres_password", None),
        "host": os.environ.get("postgres_host", None),
        "port": os.environ.get("postgres_port", None),
    }
}

postgres_handler = PostgresHandler(configuration_db, verbose=10)
postgres_handler.enable_vector_extension()

In [4]:
db_schema = "rrhh"
db_table = "general_pibot"
db_full_table_name = f"{db_schema}.{db_table}"
print(db_full_table_name)

rrhh.general_pibot


In [5]:

postgres_handler.execute_statement(f"""CREATE SCHEMA IF NOT EXISTS {db_schema};
CREATE TABLE IF NOT EXISTS {db_schema}.{db_table} (
    id int GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
    contenido_original text,
    contenido_vector vector(1536),
    vectorizacion_algoritmo VARCHAR(255),
    version varchar(3) DEFAULT '1',
	fecha_actualizacion	TIMESTAMP,
	usuario_actualizacion	VARCHAR(30),
	fecha_creacion	TIMESTAMP NOT NULL,
	usuario_creacion	VARCHAR(30) NOT NULL,
    metadata_topico VARCHAR(20)
);
""", open_and_close=True)

<cursor object at 0x000001EA6B211380; closed: 0>

In [6]:
postgres_handler.execute_statement(f"""CREATE INDEX IF NOT EXISTS {db_schema}_{db_table}_contenido_vector_idx ON {db_schema}.{db_table} 
USING hnsw (contenido_vector vector_cosine_ops) 
WITH (m = 16, ef_construction = 64)
""", open_and_close=True)

<cursor object at 0x000001EA6B2119A0; closed: 0>

In [ ]:
from Embedder import *

configuration_embedder = {
    "provider": {
        "region": os.environ.get("aws_region", None),
        "credentials": {
            "aws_access_key_id": os.environ.get("aws_access_key", None),
            "aws_secret_access_key": os.environ.get("aws_secret_key", None),
        },
    },
    "model": {"id": "amazon.titan-embed-text-v1"},
    "splitter": {
        "chunk_size": 500,
        "chunk_overlap": 20,
        "length_function": Embedder.num_tokens_from_string,
        "add_start_index": True,
    },
}

embedder = EmbedderAWS(configuration_embedder, verify=False)

In [ ]:
from aws_llm_helpers import generate_vectors



In [ ]:
import numpy as np
from datetime import datetime as datetime_CIIU
datetime_now = datetime_CIIU.now().date()
data_list_ciiu = [
    (
        row["DescLevel2"], 
        np.array(row['embeddings']), 
        configuration_embedder["model"]["id"],
        '1',
        datetime_now, 
        'ddiazpad',
        datetime_now, 
        'ddiazpad',
        row['CodeLevel2'],
    ) for index, row in cur_data_lvl2_desclevel2.iterrows()
]

postgres_handler.insert_values(
    data_list_ciiu,
    sql_columns=f"{db_full_table_name} (contenido_original, contenido_vector, vectorizacion_algoritmo, version, fecha_actualizacion, usuario_actualizacion, fecha_creacion, usuario_creacion, metadata_topico)",
    register_vector_conn=True,
    commit=True,
    open_and_close=True
)

# API Function

In [20]:
from langchain.llms.bedrock import Bedrock
from langchain.chains import LLMChain
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from bert_score import score
import os
import json
import boto3

with open(".env.json", "r") as f_in:
    env_json: dict = json.load(f_in)
    for key, value in env_json.items():
        os.environ[key] = value

aws_access_key = os.environ['aws_access_key']
aws_secret_key = os.environ['aws_secret_key']
aws_region = os.environ['aws_region']

from langchain.prompts.prompt import PromptTemplate
from langchain.memory import ConversationBufferMemory
import os
from PostgresHandler import PostgresHandler
configuration_db = {
    "postgresql": {
        "database": os.environ.get("postgres_database", None),
        "user": os.environ.get("postgres_user", None),
        "password": os.environ.get("postgres_password", None),
        "host": os.environ.get("postgres_host", None),
        "port": os.environ.get("postgres_port", None),
    }
}

postgres_handler = PostgresHandler(configuration_db, verbose=10)
postgres_handler.enable_vector_extension()

_DEFAULT_TEMPLATE = """Utiliza el siguiente contexto para responder a la pregunta
Contexto: {context}
Pregunta: {human_input}"""

rules_prompt = PromptTemplate(
    input_variables=["context", "human_input"], 
    template=_DEFAULT_TEMPLATE
)

memory = ConversationBufferMemory(memory_key="chat_history", input_key="human_input", return_messages=True)

from Embedder import *

configuration_embedder = {
    "provider": {
        "region": os.environ.get("aws_region", None),
        "credentials": {
            "aws_access_key_id": os.environ.get("aws_access_key", None),
            "aws_secret_access_key": os.environ.get("aws_secret_key", None),
        },
    },
    "model": {"id": "amazon.titan-embed-text-v1"},
    "splitter": {
        "chunk_size": 500,
        "chunk_overlap": 20,
        "length_function": Embedder.num_tokens_from_string,
        "add_start_index": True,
    },
}

embedder = EmbedderAWS(configuration_embedder, verify=False)

def handler(event, context):
   
    event_method = event["httpMethod"]
   
    if event_method != 'POST':
        return {
            'statusCode': 200,
            'body': json.dumps("Not implemented")
        }
       
    event_body = event["body"]
    prompt_data = event_body["prompt_input"]
    reference_output = event_body["respuesta_esperada"]

   
    client_bedrock_runtime = boto3.client(
        service_name="bedrock-runtime",
        region_name=aws_region,
        aws_access_key_id=aws_access_key,
        aws_secret_access_key=aws_secret_key,
        verify=False
    )
   
    model_id = 'amazon.titan-text-express-v1'
    
    titan_llm = Bedrock(
        model_id=model_id, 
        client=client_bedrock_runtime
    )

    titan_llm.model_kwargs = {
        'temperature': 0.5, 
        "maxTokenCount": 700,
        "topP": 0.9
    }
    extra_where = ""
    version = 1
    print(prompt_data)
    similar_docs = postgres_handler.get_similar_docs(
        embedder,
        user_input=prompt_data,
        schema="rrhh",
        table_name="general_pibot",
        k=3,
        custom_where=f"WHERE version = '{version}' {extra_where}",
        columns="contenido_original, metadata_topico",
        register_vector_conn=True,
        commit=True,
        open_and_close=True,
        col_name_embedding="contenido_vector",
        partition_by=None,
    )

    context = ""
    for c in similar_docs:
        context += f"{c[0]}\n"


    input_dict = {
        "context": context + str("ABCD"),
        "human_input": prompt_data
    }
    print(input_dict)

    chain = LLMChain(llm=titan_llm, prompt=rules_prompt, memory=memory, verbose=True) 
    chain_response = chain(input_dict)
    print(chain_response)
    #P, R, F1 = score(chain_response, reference_summaries, lang='es', verbose=True)
    P, R, F1 = score([chain_response["human_input"]], [reference_output], lang="es", verbose=True)

    return {
        'statusCode': 200,
        'body': json.dumps({
            'chain_response': chain_response,
            'precision': P.mean().item(),
            'recall': R.mean().item(),
            'f1': F1.mean().item(),
    })
    }


prompt_input = '¿Cuándo se agregará el valor de la cuota adicional por la inclusión de nuevos Beneficiarios según la poliza de salud de la organizacion BBB?'
reference_output = 'El valor de la cuota adicional se agregará al global de las siguientes cuotas del Contrato después de la inclusión de nuevos Beneficiarios.'

event = {
    "httpMethod": "POST",
    "body": {
        "prompt_input": prompt_input,
        "respuesta_esperada": reference_output
    }
}
lambda_response = handler(event, context=None)
print(lambda_response)
print(lambda_response.get('body'))

¿Cuándo se agregará el valor de la cuota adicional por la inclusión de nuevos Beneficiarios según la poliza de salud de la organizacion BBB?


c:\Users\usuario\anaconda3\envs\general\lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'bedrock-runtime.us-east-1.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'context': 'ABCD', 'human_input': '¿Cuándo se agregará el valor de la cuota adicional por la inclusión de nuevos Beneficiarios según la poliza de salud de la organizacion BBB?'}


> Entering new LLMChain chain...
Prompt after formatting:
Utiliza el siguiente contexto para responder a la pregunta
Contexto: ABCD
Pregunta: ¿Cuándo se agregará el valor de la cuota adicional por la inclusión de nuevos Beneficiarios según la poliza de salud de la organizacion BBB?


c:\Users\usuario\anaconda3\envs\general\lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'bedrock-runtime.us-east-1.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(



> Finished chain.
{'context': 'ABCD', 'human_input': '¿Cuándo se agregará el valor de la cuota adicional por la inclusión de nuevos Beneficiarios según la poliza de salud de la organizacion BBB?', 'chat_history': [HumanMessage(content='¿Cuándo se agregará el valor de la cuota adicional por la inclusión de nuevos Beneficiarios según la poliza de salud de la organizacion BBB?'), AIMessage(content='\n\nEl valor de la cuota adicional se agregará al mes siguiente\n')], 'text': '\n\nEl valor de la cuota adicional se agregará al mes siguiente\n'}
calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:00<00:00, 58.19it/s]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 193.07it/s]

done in 0.03 seconds, 36.02 sentences/sec


TypeError: Object of type HumanMessage is not JSON serializable

# END